# spacetrackのテスト

see https://pythonhosted.org/spacetrack/usage.html

In [1]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pandas as pd
import json
import time

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 80)

In [3]:
# userid and password are set in here
import spacetrackaccount
# print("Login with " + spacetrackaccount.userid)

In [4]:
st = SpaceTrackClient(spacetrackaccount.userid, spacetrackaccount.password)

In [5]:
def mycallback(until):
    duration = int(round(until - time.monotonic()))
    print('Sleeping for {:d} seconds.'.format(duration))

st.callback = mycallback

In [6]:
# example
#data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30',
#                     mean_motion=op.inclusive_range(0.99, 1.01),
#                     eccentricity=op.less_than(0.01), format='tle')

In [7]:
# 全衛星の最新データ(30日以内)
data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30', orderby='norad_cat_id', format='3le')

In [8]:
with open('download/tle_latest.txt', 'w') as fp:
    for line in data:
        fp.write(line + '\n')

In [9]:
# 特定の衛星の過去の全データ
data = st.tle(iter_lines=True, norad_cat_id=25544, orderby='epoch', format='tle')

In [10]:
with open('download/tle_25544_all.txt', 'w') as fp:
    for line in data:
        fp.write(line + '\n')

In [11]:
# 複数の衛星の過去の全データをjsonで取得 (まとめて取得するので、同時に多くの衛星を指定することはできない)
data = st.tle(norad_cat_id=[op.inclusive_range(1, 20)], orderby=['norad_cat_id', 'epoch'], format='json')

In [12]:
with open('download/test.json', 'w') as fp:
        fp.write(data)

In [13]:
df = pd.read_json(data, orient='records')

In [14]:
df.head(3)

,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-09 22:15:15,874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,999,3800,0.0,0.000806,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001,1958-001A,4,7615.485,110.231,2117.362,357.338,1
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-17 13:46:31,735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,999,3900,0.0,0.000830,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007,1958-001A,4,7610.615,110.125,2107.621,357.339,1
2,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-08-01 20:45:25,598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,999,4100,0.0,0.000632,0.0,33905,0 EXPLORER 1,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006,1958-001A,4,7611.820,110.151,2110.015,357.355,1


In [15]:
df.TLE_LINE1[0]

'1 00004U 58001  A 59190.92726707  .00080584 +00000-0 +00000-0 0  9997'

In [16]:
# 複数の衛星の過去の全データをjsonで取得しファイルに保存する (forで取得する)
for i in range(1, 20):
    print(i)
    data = st.tle(norad_cat_id=i, orderby=['norad_cat_id', 'epoch'], format='json')
    file = 'download/{}.json'.format(i)
    with open(file, 'w') as fp:
        fp.write(data)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [17]:
# 適当なファイルを読んでみる
df = pd.read_json('download/4.json', orient='records')
df.head(3)

,COMMENT,ORIGINATOR,NORAD_CAT_ID,OBJECT_NAME,OBJECT_TYPE,CLASSIFICATION_TYPE,INTLDES,EPOCH,EPOCH_MICROSECONDS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,FILE,TLE_LINE0,TLE_LINE1,TLE_LINE2,OBJECT_ID,OBJECT_NUMBER,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE,DECAYED
0,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-09 22:15:15,874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,999,3800,0,0.000806,0,33905,0 EXPLORER 1,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001,1958-001A,4,7615.485,110.231,2117.362,357.338,1
1,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-07-17 13:46:31,735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,999,3900,0,0.000830,0,33905,0 EXPLORER 1,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007,1958-001A,4,7610.615,110.125,2107.621,357.339,1
2,GENERATED VIA SPACETRACK.ORG API,18 SPCS,4,EXPLORER 1,PAYLOAD,U,58001A,1959-08-01 20:45:25,598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,999,4100,0,0.000632,0,33905,0 EXPLORER 1,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006,1958-001A,4,7611.820,110.151,2110.015,357.355,1
